In [ ]:
import random

def play(player1, player2, num_games, verbose=False):
    oppo_hist = []
    p1_prev_play = ""
    p2_prev_play = ""
    results = {"p1": 0, "p2": 0, "tie": 0}

    for _ in range(num_games):
        if p2_prev_play=='':
          oppo_hist.append('R')
        else:
          oppo_hist.append(p2_prev_play)
          
        p1_play = player1(p2_prev_play,oppo_hist)
        p2_play = player2(p1_prev_play,)

        if p1_play == p2_play:
            results["tie"] += 1
            winner = "Tie."
        elif (p1_play == "P" and p2_play == "R") or (p1_play == "R" and p2_play == "S") or (p1_play == "S" and p2_play == "P"):
            results["p1"] += 1
            winner = "Player 1 wins."
        elif p2_play == "P" and p1_play == "R" or p2_play == "R" and p1_play == "S" or p2_play == "S" and p1_play == "P":
            results["p2"] += 1
            winner = "Player 2 wins."

        if verbose:
            print("Player 1:", p1_play, "| Player 2:", p2_play)
            print(winner)
            if results['p2'] + results['p1']!=0:
              print(_,':',round((results['p1']*100)/(results['p2'] + results['p1']),2),'%',results)

        p1_prev_play = p1_play
        p2_prev_play = p2_play

    games_won = results['p2'] + results['p1']
    if games_won == 0:
        win_rate = 0
    else:
        win_rate = results['p1'] / games_won * 100

    print("Final results:", results)
    print(f"Player 1 win rate: {win_rate}%")

    return (win_rate)


def quincy(prev_play, counter=[0]):
    counter[0] += 1
    choices = ["R", "R", "P", "P", "S"]
    return choices[counter[0] % len(choices)]


def mrugesh(prev_opponent_play, opponent_history=[]):
    opponent_history.append(prev_opponent_play)
    last_ten = opponent_history[-10:]
    most_frequent = max(set(last_ten), key=last_ten.count)

    if most_frequent == '':
        most_frequent = "S"

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[most_frequent]


def kris(prev_opponent_play):
    if prev_opponent_play == '':
        prev_opponent_play = "R"
    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prev_opponent_play]


def abbey(prev_opponent_play,
          opponent_history=[],
          play_order=[{
              "RR": 0,
              "RP": 0,
              "RS": 0,
              "PR": 0,
              "PP": 0,
              "PS": 0,
              "SR": 0,
              "SP": 0,
              "SS": 0,
          }]):

    if not prev_opponent_play:
        prev_opponent_play = 'R'
    opponent_history.append(prev_opponent_play)

    last_two = "".join(opponent_history[-2:])
    if len(last_two) == 2:
        play_order[0][last_two] += 1

    potential_plays = [
        prev_opponent_play + "R",
        prev_opponent_play + "P",
        prev_opponent_play + "S",
    ]

    sub_order = {
        k: play_order[0][k]
        for k in potential_plays if k in play_order[0]
    }

    prediction = max(sub_order, key=sub_order.get)[-1:]

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prediction]


def human(prev_opponent_play):
    play = ""
    while play not in ['R', 'P', 'S']:
        play = input("[R]ock, [P]aper, [S]cissors? ")
        print(play)
    return play


def random_player(prev_opponent_play):
    return random.choice(['R', 'P', 'S'])

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

modelo=0

def player(prev_play, oppo_hist=[]):
    global modelo
    num=30
    if prev_play=='':
        oppo_hist=[random.choice(['R','P','S'])]
        #modelo
        capa1 = tf.keras.layers.Flatten(input_shape=[num,3])
        capa2 = tf.keras.layers.Dense(units=num*5 , activation='relu')
        capa3 = tf.keras.layers.Dense(units=num*5 , activation='relu')
        capa4 = tf.keras.layers.Dense(units=3, activation='softmax')
        modelo = tf.keras.Sequential([
            capa1,capa2,capa3,capa4
        ])
        modelo.compile(
            optimizer = 'Adam',
            metrics = ['accuracy'],
            loss = 'sparse_categorical_crossentropy'
        )

    dicRPS = {'R':[1,0,0],'P':[0,1,0],'S':[0,0,1]}
    dicRPSn = {'R':0,'P':1,'S':2}
    dic123 = {0:'R',1:'P',2:'S'}
    dicJug = {'R':'P','P':'S','S':'R'}
    guess = random.choice(['R','P','S'])

    if len(oppo_hist) > num+1:
        respuesta = []
        historia = [ oppo_hist[i-num:i] for i in range(num,len(oppo_hist)+1)][0:-1]
        respuesta = [ dicJug[i] for i in oppo_hist[num::]]
        
        training_data = [[dicRPS[j] for j in i] for i in historia]
        target_data = [[dicRPSn[j]] for j in respuesta]

        historial = modelo.fit(training_data, target_data ,epochs=75, verbose=False)

        #plt.ylim(0,1)
        #plt.plot(historial.history['loss'])
        #plt.show()

        guess = list(modelo.predict([[dicRPS[j] for j in oppo_hist[-num::]]], verbose=False)[0])
        if guess[0]>=guess[1] and guess[0]>=guess[2]:
          guess = 'R'
        elif guess[1]>=guess[0] and guess[1]>=guess[2]:
          guess = 'P'
        else:
          guess = 'S'
    return guess

In [ ]:
def test_player_vs_quincy():
    print("Testing game against quincy...")
    actual = play(player, quincy, 1000, verbose=True) >= 60
    if actual:
        print('Ok test player vs quincy.')
    else:
        print('Expected player to defeat quincy at least 60% of the time.')

def test_player_vs_abbey():
    print("Testing game against abbey...")
    actual = play(player, abbey, 1000, verbose=True) >= 60
    if actual:
        print('Ok test player vs abbey.')
    else:
        print('Expected player to defeat abbey at least 60% of the time.')

def test_player_vs_kris():
    print("Testing game against kris...")
    actual = play(player, kris, 1000, verbose=True) >= 60
    if actual:
        print('Ok test player vs kris.')
    else:
        print('Expected player to defeat kris at least 60% of the time.')

def test_player_vs_mrugesh():
    print("Testing game against mrugesh...")
    actual = play(player, mrugesh, 1000, verbose=True) >= 60
    if actual:
        print('Ok test player vs mrugesh.')
    else:
        print('Expected player to defeat mrugesh at least 60% of the time.')

In [ ]:
test_player_vs_quincy()

Testing game against quincy...
Player 1: R | Player 2: R
Tie.
Player 1: P | Player 2: P
Tie.
Player 1: S | Player 2: P
Player 1 wins.
2 : 100.0 % {'p1': 1, 'p2': 0, 'tie': 2}
Player 1: P | Player 2: S
Player 2 wins.
3 : 50.0 % {'p1': 1, 'p2': 1, 'tie': 2}
Player 1: R | Player 2: R
Tie.
4 : 50.0 % {'p1': 1, 'p2': 1, 'tie': 3}
Player 1: P | Player 2: R
Player 1 wins.
5 : 66.67 % {'p1': 2, 'p2': 1, 'tie': 3}
Player 1: R | Player 2: P
Player 2 wins.
6 : 50.0 % {'p1': 2, 'p2': 2, 'tie': 3}
Player 1: S | Player 2: P
Player 1 wins.
7 : 60.0 % {'p1': 3, 'p2': 2, 'tie': 3}
Player 1: S | Player 2: S
Tie.
8 : 60.0 % {'p1': 3, 'p2': 2, 'tie': 4}
Player 1: R | Player 2: R
Tie.
9 : 60.0 % {'p1': 3, 'p2': 2, 'tie': 5}
Player 1: R | Player 2: R
Tie.
10 : 60.0 % {'p1': 3, 'p2': 2, 'tie': 6}
Player 1: P | Player 2: P
Tie.
11 : 60.0 % {'p1': 3, 'p2': 2, 'tie': 7}
Player 1: S | Player 2: P
Player 1 wins.
12 : 66.67 % {'p1': 4, 'p2': 2, 'tie': 7}
Player 1: P | Player 2: S
Player 2 wins.
13 : 57.14 % {'p1':

In [ ]:
test_player_vs_abbey()

Testing game against abbey...
Player 1: S | Player 2: P
Player 1 wins.
0 : 100.0 % {'p1': 1, 'p2': 0, 'tie': 0}
Player 1: P | Player 2: S
Player 2 wins.
1 : 50.0 % {'p1': 1, 'p2': 1, 'tie': 0}
Player 1: R | Player 2: S
Player 1 wins.
2 : 66.67 % {'p1': 2, 'p2': 1, 'tie': 0}
Player 1: R | Player 2: R
Tie.
3 : 66.67 % {'p1': 2, 'p2': 1, 'tie': 1}
Player 1: P | Player 2: P
Tie.
4 : 66.67 % {'p1': 2, 'p2': 1, 'tie': 2}
Player 1: R | Player 2: S
Player 1 wins.
5 : 75.0 % {'p1': 3, 'p2': 1, 'tie': 2}
Player 1: R | Player 2: P
Player 2 wins.
6 : 60.0 % {'p1': 3, 'p2': 2, 'tie': 2}
Player 1: P | Player 2: P
Tie.
7 : 60.0 % {'p1': 3, 'p2': 2, 'tie': 3}
Player 1: P | Player 2: S
Player 2 wins.
8 : 50.0 % {'p1': 3, 'p2': 3, 'tie': 3}
Player 1: S | Player 2: S
Tie.
9 : 50.0 % {'p1': 3, 'p2': 3, 'tie': 4}
Player 1: S | Player 2: S
Tie.
10 : 50.0 % {'p1': 3, 'p2': 3, 'tie': 5}
Player 1: S | Player 2: S
Tie.
11 : 50.0 % {'p1': 3, 'p2': 3, 'tie': 6}
Player 1: R | Player 2: S
Player 1 wins.
12 : 57.14 

In [ ]:
test_player_vs_kris()

Testing game against kris...
Player 1: S | Player 2: P
Player 1 wins.
0 : 100.0 % {'p1': 1, 'p2': 0, 'tie': 0}
Player 1: R | Player 2: R
Tie.
1 : 100.0 % {'p1': 1, 'p2': 0, 'tie': 1}
Player 1: S | Player 2: P
Player 1 wins.
2 : 100.0 % {'p1': 2, 'p2': 0, 'tie': 1}
Player 1: P | Player 2: R
Player 1 wins.
3 : 100.0 % {'p1': 3, 'p2': 0, 'tie': 1}
Player 1: S | Player 2: S
Tie.
4 : 100.0 % {'p1': 3, 'p2': 0, 'tie': 2}
Player 1: R | Player 2: R
Tie.
5 : 100.0 % {'p1': 3, 'p2': 0, 'tie': 3}
Player 1: P | Player 2: P
Tie.
6 : 100.0 % {'p1': 3, 'p2': 0, 'tie': 4}
Player 1: R | Player 2: S
Player 1 wins.
7 : 100.0 % {'p1': 4, 'p2': 0, 'tie': 4}
Player 1: P | Player 2: P
Tie.
8 : 100.0 % {'p1': 4, 'p2': 0, 'tie': 5}
Player 1: S | Player 2: S
Tie.
9 : 100.0 % {'p1': 4, 'p2': 0, 'tie': 6}
Player 1: R | Player 2: R
Tie.
10 : 100.0 % {'p1': 4, 'p2': 0, 'tie': 7}
Player 1: P | Player 2: P
Tie.
11 : 100.0 % {'p1': 4, 'p2': 0, 'tie': 8}
Player 1: S | Player 2: S
Tie.
12 : 100.0 % {'p1': 4, 'p2': 0, 't

In [ ]:
test_player_vs_mrugesh()

Testing game against mrugesh...
Player 1: S | Player 2: R
Player 2 wins.
0 : 0.0 % {'p1': 0, 'p2': 1, 'tie': 0}
Player 1: R | Player 2: R
Tie.
1 : 0.0 % {'p1': 0, 'p2': 1, 'tie': 1}
Player 1: S | Player 2: R
Player 2 wins.
2 : 0.0 % {'p1': 0, 'p2': 2, 'tie': 1}
Player 1: S | Player 2: R
Player 2 wins.
3 : 0.0 % {'p1': 0, 'p2': 3, 'tie': 1}
Player 1: R | Player 2: R
Tie.
4 : 0.0 % {'p1': 0, 'p2': 3, 'tie': 2}
Player 1: R | Player 2: R
Tie.
5 : 0.0 % {'p1': 0, 'p2': 3, 'tie': 3}
Player 1: S | Player 2: R
Player 2 wins.
6 : 0.0 % {'p1': 0, 'p2': 4, 'tie': 3}
Player 1: S | Player 2: R
Player 2 wins.
7 : 0.0 % {'p1': 0, 'p2': 5, 'tie': 3}
Player 1: S | Player 2: R
Player 2 wins.
8 : 0.0 % {'p1': 0, 'p2': 6, 'tie': 3}
Player 1: P | Player 2: R
Player 1 wins.
9 : 14.29 % {'p1': 1, 'p2': 6, 'tie': 3}
Player 1: P | Player 2: R
Player 1 wins.
10 : 25.0 % {'p1': 2, 'p2': 6, 'tie': 3}
Player 1: R | Player 2: R
Tie.
11 : 25.0 % {'p1': 2, 'p2': 6, 'tie': 4}
Player 1: S | Player 2: R
Player 2 wins.
1